In [1]:
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#import os
import re
#import operator
#import datetime
#from collections import Counter
#from nltk import ngrams
#import gc
#import warnings
#warnings.filterwarnings('ignore')
#from pylab import rcParams
#rcParams['figure.figsize'] = 20,10
#from sklearn import metrics
#from sklearn.model_selection import cross_val_score

In [2]:
def get_length(df):
    return [len(x.split()) for x in df['request_text_edit_aware']]

def get_karma(df):
    karma = df['requester_upvotes_plus_downvotes_at_request'] + df['requester_upvotes_plus_downvotes_at_retrieval']
    return karma
    
def get_score(df):
    score = df['requester_upvotes_minus_downvotes_at_request'] + df['requester_upvotes_minus_downvotes_at_retrieval']
    return score

def get_age(df):
    return df['requester_days_since_first_post_on_raop_at_request']

def get_popularity(df):
        return df['number_of_upvotes_of_request_at_retrieval']

def get_evidentiality(df):
    urls = []
    for text in df['request_text_edit_aware']:
        url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
        urls.append(len(url))
    return urls

def get_activity(df):
    return df['requester_number_of_subreddits_at_request']

def train_model(data,model,response_col,scoring='roc_auc',cv=5):
        data_copy = data.copy()
        y = data_copy.pop(response_col)
        scores = cross_val_score(model,data_copy,y,scoring=scoring,cv=cv)
        del data_copy
        gc.collect()
        return scores.mean()

In [3]:
df = pd.read_json('train.json')

In [4]:
df['length'] = get_length(df)
df['karma'] = get_karma(df)
df['scores'] = get_score(df)
df['age']= get_age(df)
df['evidentiality'] = get_evidentiality(df)
df['activity'] = get_activity(df)
#narrative_topics = get_narrative(df,narrative=narrative)
#df['nt'] = narrative_topics
df = pd.get_dummies(df,columns=['nt'])
df['popularity']= get_popularity(df)
df['politeness']= get_politeness(df,polite_words)
df['pizza_received']= df['requester_received_pizza']
#print(df['pizza_received'])

KeyError: "['nt'] not in index"

In [35]:
df_new = df.drop([col for col in df.columns if col not in req_features],axis=1)
df_new.head(20)

,length,karma,scores,age,evidentiality,activity,nt_Craving,nt_Family,nt_Job,nt_Money,...,nt_Craving,nt_Family,nt_Job,nt_Money,nt_Student,nt_Craving,nt_Family,nt_Job,nt_Money,nt_Student
0,67,1,1,0.000000,0,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
1,16,11284,4292,0.000000,0,12,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
2,59,3,3,0.000000,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,30,157,113,0.000000,0,4,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
4,103,3620,2346,101.606505,0,11,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
5,34,2021,767,340.819329,0,21,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
6,207,28,16,0.000000,1,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
7,87,42667,15701,0.000000,0,15,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
8,47,806,442,0.000000,0,20,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
9,59,34,32,0.000000,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0


In [29]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l1')
auc_score = train_model(df_new,model=model,response_col='pizza_received',cv=2)
acc_score = train_model(df_new,model=model,response_col='pizza_received',cv=2,scoring='accuracy')

In [31]:
print(auc_score)
print(acc_score)

0.6446023481664775
0.7492574257425743
